<a href="https://colab.research.google.com/github/LeonardoVieiraGuimaraes/MiniCursoPalestra/blob/main/CNN/inference_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference com modelo treinado (Colab)
Este notebook carrega um modelo treinado (salvo no runtime ou no Google Drive) e faz inferência em uma imagem fornecida pelo usuário. Funciona no Google Colab ou em um Jupyter local.

Fluxo:
1. Montar Google Drive (opcional, se o modelo estiver salvo lá).
2. Carregar o modelo salvo (procura em locais comuns).
3. Fazer upload de uma imagem ou usar uma URL.
4. Pré-processar e prever.

## Observações
- Este notebook assume que o modelo foi salvo num dos caminhos: `/content/drive/MyDrive/simple_cnn_mnist.h5` (Drive), `simple_cnn_mnist.h5` ou `best_model.h5` (runtime).
- O modelo espera imagens 28x28 em escala de cinza, normalizadas em [0,1]. Se fornecer imagens coloridas, o notebook fará a conversão.

In [ ]:
# Célula 1 — montar Drive se estiver no Colab (opcional)
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    print('Montando Google Drive em /content/drive ...')
    drive.mount('/content/drive')
else:
    print('Não detectado Colab. Usando ambiente local.')

In [ ]:
# Célula 2 — localizar e carregar o modelo salvo
import os
from tensorflow.keras.models import load_model

candidate_paths = []
candidate_paths.append('/content/drive/MyDrive/simple_cnn_mnist.h5')
candidate_paths.append('simple_cnn_mnist.h5')
candidate_paths.append('best_model.h5')

model_path = None
for p in candidate_paths:
    if os.path.exists(p):
        model_path = p
        break

if model_path is None:
    print('Modelo não encontrado nos caminhos padrão.')
    print('Salve o modelo em Drive (MyDrive/simple_cnn_mnist.h5) ou no runtime (simple_cnn_mnist.h5) e reexecute esta célula.')
else:
    print('Carregando modelo de', model_path)
    model = load_model(model_path)
    print('Modelo carregado com sucesso.')

In [ ]:
# Célula 3 — instalar Pillow em Colab se necessário
try:
    from PIL import Image
except Exception:
    if IN_COLAB:
        import subprocess, sys
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'pillow'])
        from PIL import Image
    else:
        raise

from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

print('Pillow disponível, carregamento de imagens pronto.')

## Carregar uma imagem
Opções:
- Fazer upload direto do seu computador (Colab/Jupyter).
- Fornecer uma URL de imagem (o notebook baixa a imagem).

In [ ]:
# Célula 4 — fazer upload ou baixar por URL
image_path = None

# 1) Primeiro tentamos upload via widget (funciona no Colab/Jupyter)
try:
    from google.colab import files as colab_files
    uploaded = colab_files.upload()  # abre diálogo no Colab
    if uploaded:
        # pega o primeiro arquivo enviado
        image_path = list(uploaded.keys())[0]
        print('Arquivo enviado:', image_path)
except Exception:
    try:
        # Em Jupyter local, usar IPython widgets alternativa
        from IPython.display import display
        import ipywidgets as widgets
        uploader = widgets.FileUpload(accept='image/*', multiple=False)
        display(uploader)
        print('Use o widget acima para selecionar um arquivo e depois execute a célula novamente.')
    except Exception:
        print('Upload não disponível — forneça uma URL manualmente.')

# 2) Se não houver upload, você pode definir uma URL aqui:
image_url = ''  # exemplo: 'https://example.com/digit.png'
if image_path is None and image_url:
    print('Baixando imagem de URL...', image_url)
    try:
        import requests
        resp = requests.get(image_url)
        img_bytes = resp.content
        with open('input_image.png', 'wb') as f:
            f.write(img_bytes)
        image_path = 'input_image.png'
    except Exception as e:
        print('Falha ao baixar a imagem:', e)

print('image_path =', image_path)

In [ ]:
# Célula 4.1 — listar imagens numa pasta do Google Drive (ou local) e escolher primeira
# Defina aqui o caminho da pasta no Drive, por exemplo: '/content/drive/MyDrive/imagens'
drive_folder = '/content/drive/MyDrive/imagens'  # altere conforme necessário

def list_images_in_folder(folder):
    import os
    from IPython.display import display
    try:
,